In [38]:
import numpy as np
from keras import backend as K
from keras.objectives import mean_squared_error
def log_poisson(y_true, log_y_pred):
    return K.mean(K.exp(log_y_pred) - y_true * log_y_pred, axis=-1)

def poi_gau_mix(y_true, log_y_pred):
    return log_poisson(y_true, log_y_pred) + 0.01*mean_squared_error(y_true, K.exp(log_y_pred))

In [39]:
16*17

272

In [40]:
# from keras.layers import Embedding, Input, Dense, Convolution2D, MaxPooling2D, UpSampling2D

from keras.models import Model
from keras.constraints import MaxNorm
from keras.layers.advanced_activations import ELU
from keras.layers.normalization import BatchNormalization

from keras.models import Model, Sequential
from keras.layers import (Dropout, Activation, Dense, LSTM, Conv1D, Conv2D,
                          Convolution1D, Convolution2D,  MaxPooling2D, UpSampling2D, Flatten)
from keras.layers.core import Reshape
from keras.layers.convolutional import ZeroPadding2D

# create the model
batch_size = 256
channels = 16
sequence_len = 240000
embed_dim = 128
# input_dim = 3
# sequence_len = 50
# embed_dim = 8
dropoutrate = 0.25

input_shape = (sequence_len, channels)

print('Build model...')
mo = Sequential()

nfilt1 = 16

mo.add(Convolution1D(nb_filter = nfilt1, filter_length = 1, 
                     input_dim=channels, input_length=sequence_len, 
                     W_regularizer='l1l2', 
                     ))
# output: 3D tensor with shape: (samples, new_steps, nb_filter). steps value might have changed due to padding.
mo.add(Reshape((1, nfilt1, sequence_len)))
mo.add(BatchNormalization())
mo.add(Activation('elu'))
mo.add(Dropout(dropoutrate))


mo.add(Convolution2D(nb_filter = 4, nb_row =2, nb_col =8,
                     W_regularizer='l1l2',))
mo.add(BatchNormalization())
mo.add(ZeroPadding2D(padding=(1, 4)))
mo.add(Activation('elu'))
mo.add(MaxPooling2D((2,4)))
mo.add(Dropout(dropoutrate))


mo.add(Convolution2D(nb_filter = 4, nb_row = 8, nb_col =4,
                     W_regularizer='l1l2',))
mo.add(BatchNormalization())
mo.add(ZeroPadding2D(padding=(1, 4)))
mo.add(Activation('elu'))
mo.add(MaxPooling2D((2,4)))
mo.add(Dropout(dropoutrate))

#################
print(mo.summary())

mo.add(Flatten())
mo.add(Dense(1))
mo.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
mo.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# print(model.summary())
# # model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=3, batch_size=64)

Build model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_20 (Convolution1D) (None, 240000, 16)    272         convolution1d_input_18[0][0]     
____________________________________________________________________________________________________
reshape_16 (Reshape)             (None, 1, 16, 240000) 0           convolution1d_20[0][0]           
____________________________________________________________________________________________________
batchnormalization_43 (BatchNorma(None, 1, 16, 240000) 480000      reshape_16[0][0]                 
____________________________________________________________________________________________________
activation_45 (Activation)       (None, 1, 16, 240000) 0           batchnormalization_43[0][0]      
____________________________________________________________________________

In [41]:
filepath = "eegnet"
# mo.load_weights(filepath)

In [42]:
mo.optimizer.lr.set_value(.01)

In [43]:
from keras.callbacks import LearningRateScheduler
def scheduler(epoch):
    if epoch == 15:
        mo.optimizer.lr.set_value(.01)
    if epoch == 20:
        mo.optimizer.lr.set_value(.005)
    return float(mo.optimizer.lr.get_value())

change_lr = LearningRateScheduler(scheduler)

In [44]:
from scipy.io import loadmat
import six
def mat_to_data(path):
    mat = loadmat(path)
    names = mat['dataStruct'].dtype.names
    ndata = {n: mat['dataStruct'][n][0, 0] for n in names}
    for kk,vv in six.iteritems(ndata):
#         print(vv.shape)
        if vv.shape == (1,1):
            ndata[kk] = vv[0,0]
    return ndata

def get_label(infile):
    return infile.split(".")[-2][-1] == "0"

def test_mat_to_data(path = "./data/1_1_0.mat"):
    data = mat_to_data(path)
    label = get_label(path)

In [45]:
import os
def filegen(folder, BATCH_SIZE = 64):
    xx = []
    yy = []
    infiles = list(filter(lambda x: x.endswith("mat"), os.listdir(folder)))
    NUM_FILES = len(infiles)
    for nn, ff in enumerate(infiles):
        print(ff)
        ff = os.path.join(folder,ff)
        label = get_label(ff)
        data = mat_to_data(ff)
#         yield np.dstack(data["data"]).transpose(0,2,1), np.array([[label]])
        xx.append(data["data"])
        yy.append(label)
        if ((nn+1) % BATCH_SIZE == 0) or ((nn+1)==NUM_FILES):
            data = np.dstack(xx).transpose(2,0,1)
            label = np.array(yy)[:,np.newaxis]
#             print(data.shape, label.shape)
            xx = []
            yy = []
            yield data, label


datadir = "data/"
BATCH_SIZE = 2
gen = filegen(datadir, BATCH_SIZE = BATCH_SIZE)

for xx, yy in gen:
    print(xx.shape, yy.shape)
    pass

1_1_0.mat
1_1_1.mat
(2, 240000, 16) (2, 1)
1_2_0.mat
1_2_1.mat
(2, 240000, 16) (2, 1)


imdb example:

    X (25000 , 80)
      (num_samples, seq_len)
     
eeg:

    X (num_samples, seq_len, embedding_dim)

In [46]:
from itertools import cycle
datadir = "data/"
BATCH_SIZE = 4
gen = cycle(filegen(datadir, BATCH_SIZE = BATCH_SIZE))
# print(type(gen))
# print(gen.next())

mo.fit_generator(gen, nb_worker=1,
                 nb_epoch=300, samples_per_epoch = BATCH_SIZE,
                 callbacks=[change_lr])

Epoch 1/300
1_1_0.mat
1_1_1.mat
1_2_0.mat
1_2_1.mat
4/4 [==============================] - 17s - loss: 1.8870 - acc: 0.2500
Epoch 2/300
4/4 [==============================] - 15s - loss: 8.5659 - acc: 0.5000
Epoch 3/300
4/4 [==============================] - 13s - loss: 8.5714 - acc: 0.5000
Epoch 4/300
4/4 [==============================] - 15s - loss: 8.5754 - acc: 0.5000
Epoch 5/300
4/4 [==============================] - 15s - loss: 8.5784 - acc: 0.5000
Epoch 6/300
4/4 [==============================] - 10s - loss: 8.5803 - acc: 0.5000
Epoch 7/300
4/4 [==============================] - 12s - loss: 8.5818 - acc: 0.5000
Epoch 8/300
4/4 [==============================] - 11s - loss: 8.5822 - acc: 0.5000
Epoch 9/300
4/4 [==============================] - 11s - loss: 8.5812 - acc: 0.5000
Epoch 10/300
4/4 [==============================] - 13s - loss: 8.5795 - acc: 0.5000
Epoch 11/300
4/4 [==============================] - 11s - loss: 8.5769 - acc: 0.5000
Epoch 12/300
4/4 [================